In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hys
import sys
sys.path.append("E:/github_repo/4DSTEM_processing/")
from FDSTEM_process import *
import tifffile
import tkinter.filedialog as tkf
import time
import rigidregistration as rr
plt.rcParams['font.family'] = 'Cambria'

In [ ]:
ref_adr = tkf.askopenfilename()
print(ref_adr)
ref_data = hys.load(ref_adr).data
ref_data = np.rollaxis(ref_data, 0, 3)
print(ref_data.shape)

In [ ]:
s = rr.stackregistration.imstack(ref_data)
print(s)
s.getFFTs()

In [ ]:
masktype="gaussian"
i,j = 0, 1
n=2
s.makeFourierMask(mask=masktype,n=n)
s.show_Fourier_mask(i=i,j=j)

In [ ]:
shifts = s.findImageShifts(findMaxima='pixel',verbose=False)
s.show_Rij()
plt.show()

s.get_outliers(threshold=15)              # Set outlier threshhold
s.show_Rij()
plt.show()

s.make_corrected_Rij()    # Correct outliers using the transitivity relations
s.show_Rij_c()
plt.show()

In [ ]:
s.get_averaged_image()

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(s.average_image)
plt.show()

In [ ]:
tifffile.imwrite("aligned_ADF.tif", s.average_image)

In [ ]:
raw_adr = tkf.askopenfilenames()
print(raw_adr)

xsh = hys.load(raw_adr[0]).data
print(xsh.shape)
print(np.mean(xsh), np.std(xsh))
ysh = hys.load(raw_adr[1]).data
print(ysh.shape)
print(np.mean(ysh), np.std(ysh))

In [ ]:
s.imstack = np.rollaxis(xsh, 0, 3)
s.get_averaged_image()

xsh_aligned = s.average_image

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(xsh_aligned)
plt.show()

In [ ]:
s.imstack = np.rollaxis(ysh, 0, 3)
s.get_averaged_image()

ysh_aligned = s.average_image

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(ysh_aligned)
plt.show()

In [ ]:
print(np.mean(xsh_aligned), np.std(xsh_aligned))
print(np.mean(ysh_aligned), np.std(ysh_aligned))
print(np.mean(xsh_aligned)/np.mean(ysh_aligned))
ratio = np.mean(xsh_aligned)/np.mean(ysh_aligned)

In [ ]:
hpass_val = 1E-3
lpass_val = 1E-3

E_mag, E_field_x, E_field_y, charge_density, potential = segmented_DPC(xsh_aligned, ysh_aligned*ratio, 
                                                                    correct_rotation=True,
                                                                    n_theta=50,
                                                                    hpass=hpass_val,
                                                                    lpass=lpass_val,
                                                                    visual=True)

In [ ]:
tifffile.imwrite("aligned_iDPC.tif", potential)

In [ ]:
potentials = []

for i in range(10):
    E_mag, E_field_x, E_field_y, charge_density, potential = segmented_DPC(xsh[i], ysh[i], 
                                                                        correct_rotation=True,
                                                                        n_theta=50,
                                                                        hpass=hpass_val,
                                                                        lpass=lpass_val,
                                                                        visual=True)
    plt.show()
    potentials.append(potential)

In [ ]:
potentials = np.asarray(potentials)
data = np.rollaxis(potentials, 0, 3)

In [ ]:
s.imstack = data.copy()
s.get_averaged_image()

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(s.average_image)
plt.show()

In [ ]:
tifffile.imwrite("aligned_iDPC.tif", s.average_image)